In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pandas.core.frame import DataFrame
import math

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM,GRU
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint
from tensorflow.keras.layers import Conv2D 
from keras.utils import np_utils
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import numpy as np
import scipy.io
import h5py
import keras
import os
from keras.layers import *
from keras.layers.core import *
from keras.models import *
from keras.regularizers import *
from keras import metrics
from keras import backend as K
from keras.backend import sum, mean, max
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from keras.callbacks import EarlyStopping
#from keras_self_attention import SeqSelfAttention

In [ ]:
import json
with open('/content/drive/MyDrive/project_folder/data/basic_information/' + 'whole_train_4' + '.json','r') as json_file:
  train = json.load(json_file)
with open('/content/drive/MyDrive/project_folder/data/basic_information/' + 'whole_test_3' + '.json','r') as json_file:
  test = json.load(json_file)

In [ ]:
# replce inf with 0 in training set
train_new=[]
for i in range(len(train)):
  df=pd.DataFrame(train[i])
  df.replace(np.inf, 0, inplace=True)
  train_new.append(df)

In [ ]:
# check if there is inf in training set
inf1=[]
for i in range(len(train_new)):
  for j in range (40):
    for z in range(10):
      if math.isinf(pd.DataFrame(train_new[i]).iloc[:,j][z])== True:
        inf1.append(1)
        print(i,j,z)

len(inf1)

In [ ]:
# check if there is inf in test set
inf1=[]
for i in range(len(test)):
  for j in range (40):
    for z in range(10):
      if math.isinf(pd.DataFrame(test[i]).iloc[:,j][z])== True:
        inf1.append(1)
        print(i,j,z)

len(inf1)

In [ ]:
## split train and test
import random
random.shuffle(train_new)
random.shuffle(test)

#train = whole[:round(len(whole)*0.8)] 
#test = whole[round(len(whole)*0.8):]

train =  train_new
train_X = []
train_Y = []
#train_Y_10 = []

for i in range(len(train)):
  train_X.append(pd.DataFrame(train[i]).drop(['label'], axis = 1))
  train_Y.append(pd.DataFrame(train[i])['label'][0]) # 10 timeseries to 1 label
  #train_Y_10.append(pd.DataFrame(pd.DataFrame(train[i])['label'])) # 10 timeseries to 10 labels


test_X = []
test_Y = []
#test_Y_10 = []

for i in range(len(test)):
  test_X.append(pd.DataFrame(test[i]).drop(columns = 'label'))
  test_Y.append(pd.DataFrame(test[i])['label'][0])

In [ ]:
from keras.utils.np_utils import to_categorical
NUM_LABELS = 2
total_labels = ["positive", "negative"]

for i in range(len(train_Y)):
    if train_Y[i] == 'positive':
      train_Y[i] = 1
    else: train_Y[i] = 0

for i in range(len(test_Y)):
    if test_Y[i] == 'positive':
      test_Y[i] = 1
    else: test_Y[i] = 0

In [ ]:
# proportion
label = ['negative', 'positve']
label_1 = np.sum(np.array(train_Y))
label_0 = len(train_Y) - label_1
plt.bar([0,1], [label_0, label_1], align='center', alpha=0.7)
plt.xticks(np.arange(2), label)
plt.ylabel('Number')
plt.title('Training Set Label Distribution')
plt.show()

In [ ]:
np.mean(train_Y)

In [ ]:
def plot_confusion_matrix(y_true, y_pred, title = "Confusion Matrix",
                          cmap = plt.cm.Blues, save_flg = False):
  #classes = [str(i) for i in range(2)]
  classes = ['negative', 'positive']
  labels = range(2)
  cm = confusion_matrix(y_true, y_pred, labels=labels)
  plt.figure(figsize=(7, 6))
  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title, fontsize=20)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, fontsize=10)
  plt.yticks(tick_marks, classes, fontsize=10)
  print('Confusion matrix, without normalization')
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, cm[i, j],
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black")
  plt.ylabel('True label', fontsize=15)
  plt.xlabel('Predicted label', fontsize=15)
  if save_flg:
      plt.savefig("./confusion_matrix.png")
  plt.show()

In [ ]:
# Model 2
def get_col(col):
  def func(x):
    return x[:,:,col:col+1]
  return Lambda(func)

In [ ]:
input = Input(shape=(10,3))
lstm_units = 8
LSTM1 = LSTM(lstm_units, return_sequences=True, implementation=1)(input)
Dropout1 = Dropout(0.1)(LSTM1)
LSTM2 = LSTM(lstm_units, return_sequences=True, implementation=1)(Dropout1)
Dropout2 = Dropout(0.1)(LSTM2)
LSTM3 = LSTM(lstm_units, return_sequences=True, implementation=1)(Dropout2)
Dropout3 = Dropout(0.1)(LSTM3)
LSTMMODEL = Model(inputs=input, outputs=Dropout3)
LSTMMODEL.summary()

In [ ]:

inputs = Input(shape=(10, 40)) # 10 frames * 40 features

# Each body part concludes one angle and distance of two line sengemnts
#BODY PART 1                            
angle1=get_col(0)(inputs)
d_11=get_col(19)(inputs)
d_21=get_col(20)(inputs)
B1=concatenate([angle1, d_11,d_21], axis=-1)
print(B1)
TemporalAttention = Conv1D(1, 1, strides=1)(LSTMMODEL(B1))
TemporalAttention = Softmax(axis=-2, name='TemporalAttention1')(TemporalAttention)
out = multiply([LSTMMODEL(B1), TemporalAttention])
out = Lambda(lambda x: sum(x, axis=1, keepdims=True))(out)
Blast1 = Permute((2, 1), input_shape=(inputs, lstm_units))(out)

#BODY PART 2  
angle2=get_col(1)(inputs)
d_12=get_col(19)(inputs)
d_22=get_col(21)(inputs)
B2=concatenate([angle2, d_12,d_22], axis=-1)
TemporalAttention = Conv1D(1, 1, strides=1)(LSTMMODEL(B2))
TemporalAttention = Softmax(axis=-2, name='TemporalAttention2')(TemporalAttention)
out = multiply([LSTMMODEL(B2), TemporalAttention])
out = Lambda(lambda x: sum(x, axis=1, keepdims=True))(out)
Blast2 = Permute((2, 1), input_shape=(inputs, lstm_units))(out)

#BODY PART 3  
angle3=get_col(2)(inputs)
d_13=get_col(20)(inputs)
d_23=get_col(22)(inputs)
B3=concatenate([angle3, d_13,d_23], axis=-1)
TemporalAttention = Conv1D(1, 1, strides=1)(LSTMMODEL(B3))
TemporalAttention = Softmax(axis=-2, name='TemporalAttention3')(TemporalAttention)
out = multiply([LSTMMODEL(B3), TemporalAttention])
out = Lambda(lambda x: sum(x, axis=1, keepdims=True))(out)
Blast3 = Permute((2, 1), input_shape=(inputs, lstm_units))(out)

#BODY PART 4  
angle4=get_col(3)(inputs)
d_14=get_col(20)(inputs)
d_24=get_col(23)(inputs)
B4=concatenate([angle4, d_14,d_24], axis=-1)
TemporalAttention = Conv1D(1, 1, strides=1)(LSTMMODEL(B4))
TemporalAttention = Softmax(axis=-2, name='TemporalAttention4')(TemporalAttention)
out = multiply([LSTMMODEL(B4), TemporalAttention])
out = Lambda(lambda x: sum(x, axis=1, keepdims=True))(out)
Blast4 = Permute((2, 1), input_shape=(inputs, lstm_units))(out)

#BODY PART 5  
angle5=get_col(4)(inputs)
d_15=get_col(22)(inputs)
d_25=get_col(24)(inputs)
B5=concatenate([angle5, d_15,d_25], axis=-1)
TemporalAttention = Conv1D(1, 1, strides=1)(LSTMMODEL(B5))
TemporalAttention = Softmax(axis=-2, name='TemporalAttention5')(TemporalAttention)
out = multiply([LSTMMODEL(B5), TemporalAttention])
out = Lambda(lambda x: sum(x, axis=1, keepdims=True))(out)
Blast5 = Permute((2, 1), input_shape=(inputs, lstm_units))(out)

#BODY PART 6  
angle6=get_col(5)(inputs)
d_16=get_col(23)(inputs)
d_26=get_col(25)(inputs)
B6=concatenate([angle6, d_16,d_26], axis=-1)
TemporalAttention = Conv1D(1, 1, strides=1)(LSTMMODEL(B6))
TemporalAttention = Softmax(axis=-2, name='TemporalAttention6')(TemporalAttention)
out = multiply([LSTMMODEL(B6), TemporalAttention])
out = Lambda(lambda x: sum(x, axis=1, keepdims=True))(out)
Blast6 = Permute((2, 1), input_shape=(inputs, lstm_units))(out)

#BODY PART 7  
angle7=get_col(6)(inputs)
d_17=get_col(26)(inputs)
d_27=get_col(27)(inputs)
B7=concatenate([angle7, d_17,d_27], axis=-1)
TemporalAttention = Conv1D(1, 1, strides=1)(LSTMMODEL(B7))
TemporalAttention = Softmax(axis=-2, name='TemporalAttention7')(TemporalAttention)
out = multiply([LSTMMODEL(B7), TemporalAttention])
out = Lambda(lambda x: sum(x, axis=1, keepdims=True))(out)
Blast7 = Permute((2, 1), input_shape=(inputs, lstm_units))(out)

#BODY PART 8  
angle8=get_col(7)(inputs)
d_18=get_col(26)(inputs)
d_28=get_col(28)(inputs)
B8=concatenate([angle8, d_18,d_28], axis=-1)
TemporalAttention = Conv1D(1, 1, strides=1)(LSTMMODEL(B8))
TemporalAttention = Softmax(axis=-2, name='TemporalAttention8')(TemporalAttention)
out = multiply([LSTMMODEL(B8), TemporalAttention])
out = Lambda(lambda x: sum(x, axis=1, keepdims=True))(out)
Blast8 = Permute((2, 1), input_shape=(inputs,lstm_units))(out)

#BODY PART 9  
angle9=get_col(8)(inputs)
d_19=get_col(27)(inputs)
d_29=get_col(29)(inputs)
B9=concatenate([angle9, d_19,d_29], axis=-1)
TemporalAttention = Conv1D(1, 1, strides=1)(LSTMMODEL(B9))
TemporalAttention = Softmax(axis=-2, name='TemporalAttention9')(TemporalAttention)
out = multiply([LSTMMODEL(B9), TemporalAttention])
out = Lambda(lambda x: sum(x, axis=1, keepdims=True))(out)
Blast9 = Permute((2, 1), input_shape=(inputs, lstm_units))(out)

#BODY PART 10  
angle10=get_col(9)(inputs)
d_110=get_col(28)(inputs)
d_210=get_col(30)(inputs)
B10=concatenate([angle10, d_110,d_210], axis=-1)
TemporalAttention = Conv1D(1, 1, strides=1)(LSTMMODEL(B10))
TemporalAttention = Softmax(axis=-2, name='TemporalAttention10')(TemporalAttention)
out = multiply([LSTMMODEL(B10), TemporalAttention])
out = Lambda(lambda x: sum(x, axis=1, keepdims=True))(out)
Blast10 = Permute((2, 1), input_shape=(inputs, lstm_units))(out)

#BODY PART 11  
angle11=get_col(10)(inputs)
d_111=get_col(29)(inputs)
d_211=get_col(31)(inputs)
B11=concatenate([angle11, d_111,d_211], axis=-1)
TemporalAttention = Conv1D(1, 1, strides=1)(LSTMMODEL(B11))
TemporalAttention = Softmax(axis=-2, name='TemporalAttention11')(TemporalAttention)
out = multiply([LSTMMODEL(B11), TemporalAttention])
out = Lambda(lambda x: sum(x, axis=1, keepdims=True))(out)
Blast11 = Permute((2, 1), input_shape=(inputs, lstm_units))(out)

#BODY PART 12  
angle12=get_col(11)(inputs)
d_112=get_col(30)(inputs)
d_212=get_col(32)(inputs)
B12=concatenate([angle12, d_112,d_212], axis=-1)
TemporalAttention = Conv1D(1, 1, strides=1)(LSTMMODEL(B12))
TemporalAttention = Softmax(axis=-2, name='TemporalAttention12')(TemporalAttention)
out = multiply([LSTMMODEL(B12), TemporalAttention])
out = Lambda(lambda x: sum(x, axis=1, keepdims=True))(out)
Blast12 = Permute((2, 1), input_shape=(inputs, lstm_units))(out)

#BODY PART 13  
angle13=get_col(12)(inputs)
d_113=get_col(33)(inputs)
d_213=get_col(20)(inputs)
B13=concatenate([angle13, d_113,d_213], axis=-1)
TemporalAttention = Conv1D(1, 1, strides=1)(LSTMMODEL(B13))
TemporalAttention = Softmax(axis=-2, name='TemporalAttention13')(TemporalAttention)
out = multiply([LSTMMODEL(B13), TemporalAttention])
out = Lambda(lambda x: sum(x, axis=1, keepdims=True))(out)
Blast13 = Permute((2, 1), input_shape=(inputs, lstm_units))(out)

#BODY PART 14  
angle14=get_col(13)(inputs)
d_114=get_col(34)(inputs)
d_214=get_col(21)(inputs)
B14=concatenate([angle14, d_114,d_214], axis=-1)
TemporalAttention = Conv1D(1, 1, strides=1)(LSTMMODEL(B14))
TemporalAttention = Softmax(axis=-2, name='TemporalAttention14')(TemporalAttention)
out = multiply([LSTMMODEL(B14), TemporalAttention])
out = Lambda(lambda x: sum(x, axis=1, keepdims=True))(out)
Blast14 = Permute((2, 1), input_shape=(inputs, lstm_units))(out)

#BODY PART 15  
angle15=get_col(14)(inputs)
d_115=get_col(35)(inputs)
d_215=get_col(36)(inputs)
B15=concatenate([angle15, d_115,d_215], axis=-1)
TemporalAttention = Conv1D(1, 1, strides=1)(LSTMMODEL(B15))
TemporalAttention = Softmax(axis=-2, name='TemporalAttention15')(TemporalAttention)
out = multiply([LSTMMODEL(B15), TemporalAttention])
out = Lambda(lambda x: sum(x, axis=1, keepdims=True))(out)
Blast15 = Permute((2, 1), input_shape=(inputs, lstm_units))(out)

#BODY PART 16  
angle16=get_col(15)(inputs)
d_116=get_col(35)(inputs)
d_216=get_col(37)(inputs)
B16=concatenate([angle16, d_116,d_216], axis=-1)
TemporalAttention = Conv1D(1, 1, strides=1)(LSTMMODEL(B16))
TemporalAttention = Softmax(axis=-2, name='TemporalAttention16')(TemporalAttention)
out = multiply([LSTMMODEL(B16), TemporalAttention])
out = Lambda(lambda x: sum(x, axis=1, keepdims=True))(out)
Blast16 = Permute((2, 1), input_shape=(inputs, lstm_units))(out)

#BODY PART 17  
angle17=get_col(16)(inputs)
d_117=get_col(35)(inputs)
d_217=get_col(38)(inputs)
B17=concatenate([angle17, d_117,d_217], axis=-1)
TemporalAttention = Conv1D(1, 1, strides=1)(LSTMMODEL(B17))
TemporalAttention = Softmax(axis=-2, name='TemporalAttention17')(TemporalAttention)
out = multiply([LSTMMODEL(B17), TemporalAttention])
out = Lambda(lambda x: sum(x, axis=1, keepdims=True))(out)
Blast17 = Permute((2, 1), input_shape=(inputs, lstm_units))(out)

#BODY PART 18  
angle18=get_col(17)(inputs)
d_118=get_col(35)(inputs)
d_218=get_col(39)(inputs)
B18=concatenate([angle18, d_118,d_218], axis=-1)
TemporalAttention = Conv1D(1, 1, strides=1)(LSTMMODEL(B18))
TemporalAttention = Softmax(axis=-2, name='TemporalAttention18')(TemporalAttention)
out = multiply([LSTMMODEL(B18), TemporalAttention])
out = Lambda(lambda x: sum(x, axis=1, keepdims=True))(out)
Blast18 = Permute((2, 1), input_shape=(inputs, lstm_units))(out)

#BODY PART 19  
angle19=get_col(18)(inputs)
d_119=get_col(19)(inputs)
d_219=get_col(26)(inputs)
B19=concatenate([angle19, d_119,d_219], axis=-1)
TemporalAttention = Conv1D(1, 1, strides=1)(LSTMMODEL(B19))
TemporalAttention = Softmax(axis=-2, name='TemporalAttention19')(TemporalAttention)
out = multiply([LSTMMODEL(B19), TemporalAttention])
out = Lambda(lambda x: sum(x, axis=1, keepdims=True))(out)
Blast19 = Permute((2, 1), input_shape=(inputs, lstm_units))(out)

DATA = concatenate([Blast1, Blast2, Blast3, Blast4, Blast5, Blast6, Blast7, Blast8,
                        Blast9, Blast10, Blast11, Blast12, Blast13,Blast14,Blast15,Blast16,Blast17,Blast18,Blast19
                        ], axis=2)
#DATA=Blast
# Handy and sufficient Bodily Attention Module
#a = Dense(BodyNum, activation='tanh')(DATA)
a = Dense(19, activation='tanh')(DATA)
#a = Dense(BodyNum, activation='softmax', name='bodyattention')(a)
a = Dense(19, activation='softmax', name='bodyattention')(a)
attentionresult = multiply([DATA, a])
attentionresult = Flatten()(attentionresult)
output = Dense(2, activation='softmax')(attentionresult)

TBMODEL = Model(inputs=inputs, outputs=output)
TBMODEL.summary()

In [ ]:
ada = keras.optimizers.Adam(lr=0.001)
TBMODEL.compile(loss='sparse_categorical_crossentropy',
                        optimizer=ada,
                        metrics=['accuracy'])

In [ ]:
# exponential learning rate
initial_learning_rate = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100,
    decay_rate=0.95,
    staircase=True)

TBMODEL.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=lr_schedule),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
TBMODEL.save_weights('model.h5')


In [ ]:
tensorboard3 = keras.callbacks.TensorBoard(run_logdir) 
checkpoint3 = keras.callbacks.ModelCheckpoint(file_dir + 'TBMODEL.h5',save_best_only=True)
History3 = TBMODEL.fit(np.asarray(train_X), np.asarray(train_Y),
                    epochs=200,
                    validation_data=(np.asarray(test_X),np.asarray(test_Y)),callbacks=[checkpoint3,tensorboard3])

In [ ]:
tensorboard3 = keras.callbacks.TensorBoard(run_logdir) 
checkpoint3 = keras.callbacks.ModelCheckpoint(file_dir + 'TBMODEL.h5',save_best_only=True)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100, min_delta=0.0001,restore_best_weights=True)
TBMODEL.load_weights('model.h5')
History3 = TBMODEL.fit(np.asarray(train_X), np.asarray(train_Y),
                    epochs=1000,
                    validation_split = 0.1,callbacks=[es,tensorboard3])

In [ ]:
History = TBMODEL.fit(np.asarray(train_X), np.asarray(train_Y),
                    epochs=50,
                    validation_split = 0.2)

In [ ]:
Pred_model_2 = model.predict(np.asarray(test_X))
Pred_model_2 = np.argmax(Pred_model_2,1)
Pred_model_2.shape

In [ ]:
plot_confusion_matrix(np.asarray(test_Y), Pred_model_2)

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
print('Precision of Model 1: %.3f' % precision_score(test_Y, Pred_model_2)) #TP / (FP + TP)
print('Recall of Model 1: %.3f' % recall_score(test_Y, Pred_model_2)) #TP / (FN + TP)
print('Accuracy of Model 1: %.3f' % accuracy_score(test_Y, Pred_model_2)) # (TP + TN)/ (TP + FN + TN + FP)
print('F1 Score of Model 1: %.3f' % f1_score(test_Y, Pred_model_2)) # 2* Precision Score * Recall Score/ (Precision Score + Recall Score/)